In [109]:
from pinecone import Pinecone, ServerlessSpec

In [110]:
pc = Pinecone(api_key='8bc6be74-a2e6-4a20-be99-6f700a4273b0')
    
index_name = "quickstart1"

In [111]:
pinecone_index = pc.create_index(
    name=index_name,
    dimension=8192, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'da3ada5b615759cc692e3c3d949f4f8c', 'Date': 'Sat, 09 Nov 2024 19:00:54 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [99]:
from transformers import AutoModel

In [112]:
embedding_model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True)
user_query = "where is capital of finland?"
query_embeddings = embedding_model.encode(user_query).tolist()


In [101]:
result = pinecone_index.query(
    vector=query_embeddings,
    top_k=5, #this is the number of results that are returned
    include_values=False,
    include_metadata=True
)


AttributeError: 'NoneType' object has no attribute 'query'

In [102]:
result

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

In [78]:
matched_info = ' '.join(item['metadata']['text'] for item in result['matches'])
sources = [item['metadata']['source'] for item in result['matches']]
context = f"Information: {matched_info} and the sources: {sources}"
sys_prompt = f"""
Instructions:
- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
- Utilize the context provided for accurate and specific information.
- Incorporate your preexisting knowledge to enhance the depth and relevance of your response.
- Cite your sources
Context: {context}
"""

In [103]:
from groq import Groq

client = Groq(api_key='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')

chat_completion = client.chat.completions.create(

    messages=[
        {
            "role": "system",
            "content": sys_prompt
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": user_query
        }
    ],

    # The language model which will generate the completion.
    model="llama3-8b-8192",
    # and repetitive.
    temperature=0.5,
    max_tokens=8194,
    top_p=1,
    stop=None,
    stream=False,
)

print(chat_completion.choices[0].message.content)

The capital of Finland is Helsinki.


In [104]:
import pandas as pd
import fitz  # PyMuPDF for PDF extraction

# Load data from an Excel file
excel_data = pd.read_excel('/Users/gultajseyid/Desktop/Liite 3 Datahub Muutostoiveet 20240816.xlsx')
text_data = excel_data.to_string()  # Convert dataframe to string format

# Load data from a PDF file
pdf_text = ""
with fitz.open("/Users/gultajseyid/Desktop/Datahub kehitystyöryhmä 15 20240903 asialista.pdf") as pdf:
    for page in pdf:
        pdf_text += page.get_text()

In [105]:
import re

def chunk_text(text, chunk_size=500):
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    chunks.append(current_chunk.strip())
    return chunks

chunks = chunk_text(pdf_text + text_data)


In [106]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3"
)

In [107]:
import pandas as pd
import fitz  # PyMuPDF for PDF extraction
import requests
import pinecone
from langchain.vectorstores import Pinecone


embedding_model = embed

# Load and preprocess data from Excel and PDF
def load_data():
    # Load data from Excel
    excel_data = pd.read_excel('/Users/gultajseyid/Desktop/Liite 3 Datahub Muutostoiveet 20240816.xlsx')
    text_data = excel_data.to_string()  # Convert dataframe to string format

    # Load data from PDF
    pdf_text = ""
    with fitz.open("/Users/gultajseyid/Desktop/fingrid_meeting_notes.pdf") as pdf:
        for page in pdf:
            pdf_text += page.get_text()
    
    return pdf_text #+ " " + text_data

def chunk_text(text, chunk_size=300):
    """Split text into chunks for embedding."""
    import re
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        # print(sentence)
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    chunks.append(current_chunk.strip())
    return chunks

# Load data and chunk it
data = load_data()
print('a')
chunks = chunk_text(data)

index_name = "quickstart"

index = pc.Index(index_name)

print(len(chunks))

# Prepare vectors for Pinecone
for i, chunk in enumerate(chunks):
    if len(chunk) < 2:
        continue
    embedding = embedding_model.embed_query(chunk)
    print(len(embedding))
    index.upsert([(f"chunk-{i}", embedding, {"text": chunk})])

# return 0


a
51
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096


In [108]:
# Query Pinecone with a question
def query_pinecone(question):
    question_embedding = embedding_model.embed_query(question)
    results = index.query(queries=[question_embedding], top_k=3, include_metadata=True)
    return [match['metadata']['text'] for match in results['matches']]

# Example usage
question = "What news are expected on version 2.3?"
relevant_texts = query_pinecone(question)
for text in relevant_texts:
    print(text)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 09 Nov 2024 18:59:42 GMT', 'Content-Type': 'text/plain', 'Content-Length': '82', 'Connection': 'keep-alive', 'server': 'envoy'})
HTTP response body: queries[1151]: invalid value -0.022814063 for type type.googleapis.com/QueryVector
